# 뉴스 데이터 수집 (막힌 코드입니다. 접어주세요.. ㅠㅠ)

30페이지 순회하며 뉴스 url이 있는 것에 대해서면 url과 타이틀을 가져온다.

단 url에 접속하면 컨텐츠가 404인 경우가 있다. 이럴 경우 해당 raw는 삭제한다.

404가 아닌경우엔 날짜 데이터까지 수집한다.

## URL, TITLE

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 초기 URL 설정
base_url = "https://kr.investing.com/equities/nvidia-corp-news"

# 결과 저장 리스트
results = []

# 30페이지까지 순회
for page_num in range(1, 31):
    # URL 생성
    if page_num == 1:
        url = base_url
    else:
        url = f"{base_url}/{page_num}"

    # 웹페이지 가져오기
    response = requests.get(url)

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(response.content, "html.parser")

    # 타겟 요소 선택
    target_elements = soup.select("#__next > div.md\\:relative.md\\:bg-white > div.relative.flex > div.grid.flex-1.grid-cols-1.px-4.pt-5.font-sans-v2.text-\\[\\#232526\\].antialiased.xl\\:container.sm\\:px-6.md\\:grid-cols-\\[1fr_72px\\].md\\:gap-6.md\\:px-7.md\\:pt-10.md2\\:grid-cols-\\[1fr_420px\\].md2\\:gap-8.md2\\:px-8.xl\\:mx-auto.xl\\:gap-10.xl\\:px-10 > div.min-w-0 > div:nth-child(3) > ul > li > article > div > a")

    # 결과 저장
    for element in target_elements:
        href = element.get("href")
        if href:
            href_value = "https://kr.investing.com" + href
            title_value = element.text.strip()
            results.append({"url": href_value, "title": title_value})
            print(href_value, title_value)

# 데이터프레임 생성
df = pd.DataFrame(results, columns=["url", "title"])

# CSV 파일로 저장
df.to_csv("nvidia_news.csv", index_label="index")


## URL, TITLE, CONTETNS

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 초기 URL 설정
base_url = "https://kr.investing.com/equities/nvidia-corp-news"

# 결과 저장 리스트
results = []

# 30페이지까지 순회
for page_num in range(1, 31):
    # URL 생성
    if page_num == 1:
        url = base_url
    else:
        url = f"{base_url}/{page_num}"

    # 웹페이지 가져오기
    response = requests.get(url)

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(response.content, "html.parser")

    # 타겟 요소 선택
    target_elements = soup.select("#__next > div.md\\:relative.md\\:bg-white > div.relative.flex > div.grid.flex-1.grid-cols-1.px-4.pt-5.font-sans-v2.text-\\[\\#232526\\].antialiased.xl\\:container.sm\\:px-6.md\\:grid-cols-\\[1fr_72px\\].md\\:gap-6.md\\:px-7.md\\:pt-10.md2\\:grid-cols-\\[1fr_420px\\].md2\\:gap-8.md2\\:px-8.xl\\:mx-auto.xl\\:gap-10.xl\\:px-10 > div.min-w-0 > div:nth-child(3) > ul > li > article > div > a")

    # 결과 저장
    for element in target_elements:
        href = element.get("href")
        if href:
            href_value = "https://kr.investing.com" + href
            title_value = element.text.strip()

            # 기사 내용 추출
            article_response = requests.get(href_value)
            article_soup = BeautifulSoup(article_response.content, "html.parser")

            # 기사 내용 처리
            article_content = ""
            article_elements = article_soup.select("#article > div > p, #article > div > span")
            if article_elements:
                for article_element in article_elements:
                    article_content += article_element.text.strip() + " "
            else:
                # 삭제된 기사 처리
                error_element = article_soup.select("#leftColumn > div.error404")
                if error_element:
                    print(f"삭제된 기사: {href_value}")
                    continue

            results.append({"url": href_value, "title": title_value, "content": article_content})
            print(href_value, title_value, article_content)

# 데이터프레임 생성
df = pd.DataFrame(results, columns=["url", "title", "content"])

# 삭제된 기사 제외하고 CSV 파일로 저장
df = df[df["content"].notnull()]
df.to_csv("nvidia_news.csv", index_label="index")


## (최종 403 에러로 막힘..) 한국 뉴스 DATE TITLE CONTENT

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from tqdm import tqdm

# 국가 설정 차라리 전체를 if문으로 하는게 나을지도...
national_lnk = ['kr.',''] # 0: 한국 , 1: 미국
filename = ['kr_','en_'] # 0: 한국 , 1: 미국

# 오늘 날짜에 맞게 디렉터리 생성
today = datetime.datetime.now().strftime("%Y-%m-%d")
save_path = f"./datas/{today}/{filename[0]}nvidia_news.csv"

# 디렉터리 생성
import os
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 컬럼명을 지정하여 빈 데이터프레임 생성
df = pd.DataFrame(columns=['url', "date", "title", "content"])

# CSV 파일로 저장
df.to_csv(save_path, index=False)

# 초기 URL 설정
base_url = "https://{}investing.com/equities/nvidia-corp-news".format(national_lnk[0])

# 100페이지까지 순회
for page_num in tqdm(range(1, 101)):

    # 결과 저장 리스트
    results = []

    # URL 생성
    if page_num == 1:
        url = base_url
    else:
        url = f"{base_url}/{page_num}"

    # 웹페이지 가져오기
    response = requests.get(url)

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(response.content, "html.parser")

    # 타겟 요소 선택
    target_elements = soup.select("#__next > div.md\\:relative.md\\:bg-white > div.relative.flex > div.grid.flex-1.grid-cols-1.px-4.pt-5.font-sans-v2.text-\\[#232526\\].antialiased.xl\\:container.sm\\:px-6.md\\:grid-cols-\\[1fr_72px\\].md\\:gap-6.md\\:px-7.md\\:pt-10.md2\\:grid-cols-\\[1fr_420px\\].md2\\:gap-8.md2\\:px-8.xl\\:mx-auto.xl\\:gap-10.xl\\:px-10 > div.min-w-0 > div:nth-child(3) > ul > li > article > div > a")
    
    # 결과 저장
    for element in target_elements:
        try : 
            href = element.get("href")
            if href:
                href_value = "https://{}investing.com".format(national_lnk[0]) + href
                title_value = element.text.strip()

                # 기사 내용 추출
                article_response = requests.get(href_value)
                article_soup = BeautifulSoup(article_response.content, "html.parser")

                # 기사 내용 처리
                article_content = ""
                article_elements = article_soup.select("#article > div > p, #article > div > span")
                if article_elements:
                    for article_element in article_elements:
                        article_content += article_element.text.strip() + " "
                else:
                    # 삭제된 기사 처리
                    error_element = article_soup.select("#leftColumn > div.error404")
                    if error_element:
                        print(f"삭제된 기사: {href_value}")
                        continue
                
                # 줄바꿈 문자 제거
                article_content = article_content.replace("\n", " ")

                # 날짜 정보 추출
                date_element = article_soup.select_one("#__next > div.md\\:relative.md\\:bg-white > div.relative.flex > div.grid.flex-1.grid-cols-1.px-4.pt-5.font-sans-v2.text-\\[\\#232526\\].antialiased.xl\\:container.sm\\:px-6.md\\:grid-cols-\\[1fr_72px\\].md\\:gap-6.md\\:px-7.md\\:pt-10.md2\\:grid-cols-\\[1fr_420px\\].md2\\:gap-8.md2\\:px-8.xl\\:mx-auto.xl\\:gap-10.xl\\:px-10 > div.min-w-0 > div > div:nth-child(1) > div.relative.flex.flex-col > div.mx-0.mt-1 > div.mt-2.flex.flex-col.gap-2.text-xs.md\\:mt-2\\.5.md\\:gap-2\\.5 > div > div > span")
                if date_element:
                    date_value = date_element.text[3:17].strip().replace(" ", "")
                else:
                    date_value = ""

                if article_content:  # 기사 내용이 있는 경우에만 결과에 추가 *뉴스 사이트 프로구독권 결제 한 계정만 보이는 뉴스가 있기 때문.
                    results.append({"url": href_value, "date": date_value, "title": title_value, "content": article_content})
                    print(page_num, date_value, title_value, article_content)
        except : pass
        
    # 결과를 데이터프레임으로 변환
    df = pd.DataFrame(results)

    # CSV 파일에 데이터 저장
    df.to_csv(save_path, mode='a', header=False, index=False)

100%|██████████| 100/100 [00:03<00:00, 29.06it/s]


## (최종 403 에러로 막힘..) 해외 뉴스 크롤링

페이지마다 csv 업데이트 되도록 함.

In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from tqdm import tqdm

# 국가 설정 차라리 전체를 if문으로 하는게 나을지도...
national_lnk = ['kr.',''] # 0: 한국 , 1: 미국
filename = ['kr_','en_'] # 0: 한국 , 1: 미국

# 오늘 날짜에 맞게 디렉터리 생성
today = datetime.datetime.now().strftime("%Y-%m-%d")
save_path = f"./datas/{today}/{filename[1]}nvidia_news.csv"

# 디렉터리 생성
import os
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 컬럼명을 지정하여 빈 데이터프레임 생성
df = pd.DataFrame(columns=['url', "date", "title", "content"])

# CSV 파일로 저장
df.to_csv(save_path, index=False)

# 초기 URL 설정
base_url = "https://{}investing.com/equities/nvidia-corp-news".format(national_lnk[1])

# 페이지까지 순회
for page_num in tqdm(range(1, 2)):
    # 결과 저장 리스트
    results = []

    # URL 생성
    if page_num == 1:
        url = base_url
    else:
        url = f"{base_url}/{page_num}"

    headers = {
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Cache-Control": "no-cache",
        "Connection": "Upgrade",
        "Host": "streaming.forexpros.com",
        "Origin": "https://www.investing.com",
        "Pragma": "no-cache",
        "Sec-Websocket-Extensions": "permessage-deflate; client_max_window_bits",
        "Sec-Websocket-Key": "yPUXVYuGSS8jh2A35n/E4w==",
        "Sec-Websocket-Version": "13",
        "Upgrade": "websocket",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
    }

    # 웹페이지 가져오기
    response = requests.get(url, headers=headers)
    print(response)
    """
    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(response.content, "html.parser")

    # 타겟 요소 선택
    target_elements = soup.select("#__next > div.md\\:relative.md\\:bg-white > div.relative.flex > div.grid.flex-1.grid-cols-1.px-4.pt-5.font-sans-v2.text-\\[\\#232526\\].antialiased.xl\\:container.sm\\:px-6.md\\:grid-cols-\\[1fr_72px\\].md\\:gap-6.md\\:px-7.md\\:pt-10.md2\\:grid-cols-\\[1fr_420px\\].md2\\:gap-8.md2\\:px-8.xl\\:mx-auto.xl\\:gap-10.xl\\:px-10 > div.min-w-0 > div:nth-child(3) > ul > li > article > div > a")

    # 결과 저장
    for element in target_elements:
        try:
            href = element.get("href")
            if href:
                href_value = "https://{}investing.com".format(national_lnk[1]) + href
                title_value = element.text.strip()

                # 기사 내용 추출
                article_response = requests.get(href_value)
                article_soup = BeautifulSoup(article_response.content, "html.parser")

                # 기사 내용 처리
                article_content = ""
                article_elements = article_soup.select("#article > div > p, #article > div > span")
                if article_elements:
                    for article_element in article_elements:
                        article_content += article_element.text.strip() + " "
                else:
                    # 삭제된 기사 처리
                    error_element = article_soup.select("#leftColumn > div.error404")
                    if error_element:
                        print(f"삭제된 기사: {href_value}")
                        continue

                # 줄바꿈 문자 제거
                article_content = article_content.replace("\n", " ")

                # 날짜 정보 추출
                date_element = article_soup.select_one("#__next > div.md\:relative.md\:bg-white > div.relative.flex > div.grid.flex-1.grid-cols-1.px-4.pt-5.font-sans-v2.text-\[\#232526\].antialiased.xl\:container.sm\:px-6.md\:grid-cols-\[1fr_72px\].md\:gap-6.md\:px-7.md\:pt-10.md2\:grid-cols-\[1fr_420px\].md2\:gap-8.md2\:px-8.xl\:mx-auto.xl\:gap-10.xl\:px-10 > div.min-w-0 > div > div > div.relative.flex.flex-col > div.mx-0.mt-1 > div.mt-2.flex.flex-col.gap-2.text-xs.md\:mt-2\.5.md\:gap-2\.5 > div > div:nth-child(2) > span:nth-child(2)")
                if date_element:
                    date_str = date_element.text[8:18].strip().replace(" ", "")
                    date_value = datetime.datetime.strptime(date_str, "%m/%d/%Y").strftime("%Y-%m-%d")
                else:
                    date_element = article_soup.select_one("#__next > div.md\:relative.md\:bg-white > div.relative.flex > div.grid.flex-1.grid-cols-1.px-4.pt-5.font-sans-v2.text-\[\#232526\].antialiased.xl\:container.sm\:px-6.md\:grid-cols-\[1fr_72px\].md\:gap-6.md\:px-7.md\:pt-10.md2\:grid-cols-\[1fr_420px\].md2\:gap-8.md2\:px-8.xl\:mx-auto.xl\:gap-10.xl\:px-10 > div.min-w-0 > div > div > div.relative.flex.flex-col > div.mx-0.mt-1 > div.mt-2.flex.flex-col.gap-2.text-xs.md\:mt-2\.5.md\:gap-2\.5 > div > div > span")
                    date_str = date_element.text[9:20].strip().replace(" ", "")
                    date_value = datetime.datetime.strptime(date_str, "%m/%d/%Y").strftime("%Y-%m-%d")

                if article_content:  # 기사 내용이 있는 경우에만 결과에 추가 *뉴스 사이트 프로구독권 결제 한 계정만 보이는 뉴스가 있기 때문.
                    results.append({"url": href_value, "date": date_value, "title": title_value, "content": article_content})
                    print(page_num, date_value, title_value, article_content)
        except : pass

    # 결과를 데이터프레임으로 변환
    df = pd.DataFrame(results)

    # CSV 파일에 데이터 저장
    df.to_csv(save_path, mode='a', header=False, index=False) """

 12%|█▏        | 12/100 [00:00<00:01, 58.53it/s]

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>


 24%|██▍       | 24/100 [00:00<00:01, 58.45it/s]

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>


 37%|███▋      | 37/100 [00:00<00:01, 59.82it/s]

...

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>


 97%|█████████▋| 97/100 [00:01<00:00, 59.38it/s]

<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>


100%|██████████| 100/100 [00:01<00:00, 58.63it/s]

<Response [403]>
<Response [403]>
